In [1]:
#import libraries
#conda install -c r r-essentials
library(tidyverse)
library(stringr)
library(dplyr)
library(ggplot2)
library(ggmap) #conda install -c conda-forge r-ggmap
library(sf) #conda install -c conda-forge r-sf
library(reshape2)
library(shiny)
library(shinythemes) #conda install -c conda-forge r-shinythemes
library(plotly)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'ggmap' was built under R version 3.6.3"Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.
Please cite ggmap if you use it! See citation("ggmap") for details.
Warning message:
"package 'sf' was built under R version 3.6.3"Linking to GEOS 3.9.0, GDAL 3.2.1, PROJ 7.2.1

Attaching package: 'reshape2'

The following obj

In [2]:
#import game sales data
raw_sales <- read_csv('./vgsales.csv')
head(raw_sales)

Parsed with column specification:
cols(
  Rank = col_double(),
  Name = col_character(),
  Platform = col_character(),
  Year = col_character(),
  Genre = col_character(),
  Publisher = col_character(),
  NA_Sales = col_double(),
  EU_Sales = col_double(),
  JP_Sales = col_double(),
  Other_Sales = col_double(),
  Global_Sales = col_double()
)


Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
6,Tetris,GB,1989,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26


In [3]:
#sort the sales by year
raw_sales <- raw_sales[order(raw_sales$Year),]
head(raw_sales)

Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
259,Asteroids,2600,1980,Shooter,Atari,4.00,0.26,0,0.05,4.31
545,Missile Command,2600,1980,Shooter,Atari,2.56,0.17,0,0.03,2.76
1768,Kaboom!,2600,1980,Misc,Activision,1.07,0.07,0,0.01,1.15
1971,Defender,2600,1980,Misc,Atari,0.99,0.05,0,0.01,1.05
2671,Boxing,2600,1980,Fighting,Activision,0.72,0.04,0,0.01,0.77
4027,Ice Hockey,2600,1980,Sports,Activision,0.46,0.03,0,0.01,0.49


In [4]:
#check if there is any null
colnames(raw_sales)[apply(raw_sales, 2, anyNA)]

character(0)

In [5]:
#import population data for each continent
continent_population <- read_csv('./continent_population.csv')
head(continent_population)

Parsed with column specification:
cols(
  Name = col_character(),
  Pop = col_double(),
  WorldPop = col_double(),
  GrowthRate = col_double(),
  PopDensity = col_double()
)


Name,Pop,WorldPop,GrowthRate,PopDensity
Asia,4678445024,0.5941,0.0083,145.5730
Africa,1373480428,0.1744,0.0245,45.3024
Europe,748787739,0.0951,0.0002,32.5236
North America,596564730,0.0758,0.0076,24.6065
South America,434260151,0.0551,0.0081,24.3456
Oceania,43219970,0.0055,0.0127,5.0756


In [6]:
#import population data for each country
country_population <- read_csv('./country_population.csv')
head(country_population)

Parsed with column specification:
cols(
  Country = col_character(),
  Population = col_double(),
  `Yearly Change` = col_character(),
  `Net Change` = col_double(),
  Density = col_double(),
  `Land Area` = col_double(),
  Migrants = col_double(),
  `Fert. Rate` = col_character(),
  `Med. Age` = col_character(),
  `Urban Pop` = col_character(),
  `World Share` = col_character()
)


Country,Population,Yearly Change,Net Change,Density,Land Area,Migrants,Fert. Rate,Med. Age,Urban Pop,World Share
China,1440297825,0.39%,5540090,153,9388211,-348399,1.7,38,61%,18.47%
India,1382345085,0.99%,13586631,464,2973190,-532687,2.2,28,35%,17.70%
United States,331341050,0.59%,1937734,36,9147420,954806,1.8,38,83%,4.25%
Indonesia,274021604,1.07%,2898047,151,1811570,-98955,2.3,30,56%,3.51%
Pakistan,221612785,2.00%,4327022,287,770880,-233379,3.6,23,35%,2.83%
Brazil,212821986,0.72%,1509890,25,8358140,21200,1.7,33,88%,2.73%


In [7]:
#choose useful columns
country_population <- country_population[c(1,2,5)]
head(country_population)

Country,Population,Density
China,1440297825,153
India,1382345085,464
United States,331341050,36
Indonesia,274021604,151
Pakistan,221612785,287
Brazil,212821986,25


In [8]:
#compute the normalized game sales
actual_sales <- raw_sales[-1] %>% rename('PAL_Sales' = 'EU_Sales')

NA_population <- as.numeric(continent_population[4,2]/1e9)
JP_population <- as.numeric(country_population[11,2]/1e9)
PAL_population <- as.numeric(sum(continent_population[,2])/1e9-NA_population-JP_population)

normalized_sales <- actual_sales[-9]
normalized_sales$NA_Sales <- normalized_sales$NA_Sales/NA_population
normalized_sales$JP_Sales <- normalized_sales$JP_Sales/JP_population
normalized_sales$PAL_Sales <- normalized_sales$PAL_Sales/PAL_population
normalized_sales$Global_Sales <- rowSums(normalized_sales[c(6:8)], na.rm = TRUE)

head(normalized_sales)

Name,Platform,Year,Genre,Publisher,NA_Sales,PAL_Sales,JP_Sales,Global_Sales
Asteroids,2600,1980,Shooter,Atari,6.7050561,0.036354556,0,6.7414107
Missile Command,2600,1980,Shooter,Atari,4.2912359,0.023770287,0,4.3150062
Kaboom!,2600,1980,Misc,Activision,1.7936025,0.009787765,0,1.8033903
Defender,2600,1980,Misc,Atari,1.6595014,0.006991261,0,1.6664927
Boxing,2600,1980,Fighting,Activision,1.2069101,0.005593009,0,1.2125031
Ice Hockey,2600,1980,Sports,Activision,0.7710815,0.004194756,0,0.7752762


In [9]:
#covert the normalized sales from wide to long
normalized_sales_long <- normalized_sales %>% group_by(Year) %>% summarise('NA_Sales' = sum(NA_Sales),
                                                                          'PAL_Sales' = sum(PAL_Sales),
                                                                          'JP_Sales' = sum(JP_Sales),
                                                                          'Global_Sales' = sum(Global_Sales))
normalized_sales_long <- melt(normalized_sales_long, id = 'Year', measure = c('NA_Sales', 'PAL_Sales', 'JP_Sales', 'Global_Sales'))
normalized_sales_long <- normalized_sales_long %>% rename('Region' = 'variable') %>% filter((Year >= 1979) & (Year <= 2019))
normalized_sales_long$Year = as.numeric(as.character(normalized_sales_long$Year))
head(normalized_sales_long)

Year,Region,value
1980,NA_Sales,17.75164
1981,NA_Sales,55.98722
1982,NA_Sales,45.12503
1983,NA_Sales,13.00781
1984,NA_Sales,55.78607
1985,NA_Sales,56.54039


In [10]:
#import revenues data for each country
raw_revenues <- read_csv('./region_revenues.csv')
# manually alter null cell
raw_revenues[2,3] = '22.29'
head(raw_revenues)

Warning message:
"Missing column names filled in: 'X6' [6]"Parsed with column specification:
cols(
  `Region / Country` = col_character(),
  `2013` = col_character(),
  `2012` = col_character(),
  `2011` = col_character(),
  `2010` = col_character(),
  X6 = col_logical()
)


Region / Country,2013,2012,2011,2010,X6
Asia-Pacific,$49.623 billion,$44.063 billion,$42.358 billion,$38.77 billion,NA
Japan,$22.29 billion,22.29,$23.04 billion,$21.13 billion,NA
China,$14 billion[47],$9.8 billion[48],$7.3 billion[49][50],$6.9 billion[51],NA
South Korea,$9.64 billion[53],$9.23 billion[54],$9.082 billion[55][56],$7.784 billion[55][57],NA
Australia,$2 billion[59],$1.16 billion[60],$1.5 billion[60],$1.67 billion[61],NA
Taiwan,$825 million,$773 million,$713 million,$660 million,NA


In [11]:
#remove the last null column
clean_revenues = raw_revenues[-6]
#remove dollar signs and citations
clean_revenues['2013'] <- str_extract(clean_revenues$'2013', "(?<=\\$)\\d.+(?=\\ )")%>%as.numeric()
clean_revenues['2012'] <- str_extract(clean_revenues$'2012', "(?<=\\$)\\d.+(?=\\ )")%>%as.numeric()
clean_revenues['2011'] <- str_extract(clean_revenues$'2011', "(?<=\\$)\\d.+(?=\\ )")%>%as.numeric()
clean_revenues['2010'] <- str_extract(clean_revenues$'2010', "(?<=\\$)\\d.+(?=\\ )")%>%as.numeric()

head(clean_revenues)

Region / Country,2013,2012,2011,2010
Asia-Pacific,49.623,44.063,42.358,38.770
Japan,22.290,NA,23.040,21.130
China,14.000,9.800,7.300,6.900
South Korea,9.640,9.230,9.082,7.784
Australia,NA,1.160,1.500,1.670
Taiwan,825.000,773.000,713.000,660.000


In [12]:
#standardize the revenues to million dollars
for (i in 1:nrow(clean_revenues)) {
  for (j in 2:5) {
    check = grepl("million", raw_revenues[i,j]%>%as.character(), fixed=TRUE)
    if (check==FALSE) {
      clean_revenues[i, j] <- clean_revenues[i, j]*1000
    }
  }
}
head(clean_revenues)

Region / Country,2013,2012,2011,2010
Asia-Pacific,49623,44063,42358,38770
Japan,22290,NA,23040,21130
China,14000,9800,7300,6900
South Korea,9640,9230,9082,7784
Australia,NA,1160,1500,1670
Taiwan,825,773,713,660


In [13]:
#manually alter null cells
clean_revenues[2,3] = 22.29
clean_revenues[5,2] = 2
clean_revenues[11,3] = 4
clean_revenues[21,3] = 2
clean_revenues[21,4] = 2

head(clean_revenues)

Region / Country,2013,2012,2011,2010
Asia-Pacific,49623,44063.00,42358,38770
Japan,22290,22.29,23040,21130
China,14000,9800.00,7300,6900
South Korea,9640,9230.00,9082,7784
Australia,2,1160.00,1500,1670
Taiwan,825,773.00,713,660


In [14]:
#create new dataframe for revenues by continents
continent_revenues <- clean_revenues %>%
  slice(1, 9, 16, 20, 23) %>%
  select('Region / Country', '2010', '2011', '2012', '2013') %>% 
  rename('Continent' = 'Region / Country')
head(continent_revenues)

Continent,2010,2011,2012,2013
Asia-Pacific,38770,42358,44063,49623
Europe,20660,21300,21300,20000
North America,20490,20700,20700,22800
Latin America,4740,5400,5400,3900
Middle East,1200,1983,2600,2600


In [15]:
#create new dataframe for revenues by countries
country_revenues <- clean_revenues %>%
  slice(-1, -9, -16, -19, -20, -23) %>%
  select('Region / Country', '2010', '2011', '2012', '2013') %>% 
  rename('Country' = 'Region / Country')

In [16]:
#add a new column to compute the sum of revenues
country_revenues$Total <- rowSums(country_revenues[c(2:5)], na.rm = TRUE)
head(country_revenues)

Country,2010,2011,2012,2013,Total
Japan,21130,23040,22.29,22290,66482.29
China,6900,7300,9800.00,14000,38000.00
South Korea,7784,9082,9230.00,9640,35736.00
Australia,1670,1500,1160.00,2,4332.00
Taiwan,660,713,773.00,825,2971.00
Indonesia,396,429,463.00,492,1780.00


In [17]:
#check if there is any null
colnames(country_revenues)[apply(country_revenues, 2, anyNA)]

[1] "2010" "2013"

In [18]:
#remove null cell
country_revenues <- country_revenues[complete.cases(country_revenues), ]

In [19]:
#combine country_revenues and country_population
country_revenues <- merge(country_revenues, country_population, by = 'Country')
head(country_revenues)

Country,2010,2011,2012,2013,Total,Population,Density
Australia,1670,1500,1160,2,4332,25550683,3
Brazil,391,2,2,1400,1795,212821986,25
Canada,1682,2171,2773,2300,8926,37799407,4
China,6900,7300,9800,14000,38000,1440297825,153
France,3416,3352,4,6900,13672,65298930,119
Germany,2659,2757,3360,3700,12476,83830972,240


In [20]:
#normalize the revenues by population
normalized_revenues <- country_revenues
normalized_revenues[c(2:6)] <- round(normalized_revenues[c(2:6)]/normalized_revenues$Population*1e9, 2)
normalized_revenues <- normalized_revenues %>% rename_at(vars(colnames(normalized_revenues[c(2:5)])), ~c('2010', '2011', '2012', '2013'))
head(normalized_revenues)

Country,2010,2011,2012,2013,Total,Population,Density
Australia,65360.29,58706.85,45399.96,78.28,169545.37,25550683,3
Brazil,1837.22,9.40,9.40,6578.27,8434.28,212821986,25
Canada,44498.05,57434.76,73360.94,60847.52,236141.27,37799407,4
China,4790.68,5068.40,6804.15,9720.21,26383.43,1440297825,153
France,52313.26,51333.15,61.26,105667.89,209375.56,65298930,119
Germany,31718.59,32887.61,40080.65,44136.43,148823.28,83830972,240


In [21]:
#rename within the dataframe for merging
world <- map_data("world") %>% rename('Country' = 'region')
world$Country[which(world$Country == 'USA')] = 'United States'
world$Country[which(world$Country == 'UK')] = 'United Kingdom'
world <- world[-6]
head(world)


Attaching package: 'maps'

The following object is masked from 'package:purrr':

    map



long,lat,group,order,Country
-69.89912,12.45200,1,1,Aruba
-69.89571,12.42300,1,2,Aruba
-69.94219,12.43853,1,3,Aruba
-70.00415,12.50049,1,4,Aruba
-70.06612,12.54697,1,5,Aruba
-70.05088,12.59707,1,6,Aruba


In [22]:
#merge actual_revenue and world to get coordinate
actual_revenues <- merge(country_revenues, world, by = 'Country', all='True')
actual_revenues <- actual_revenues[order(actual_revenues$order),]
actual_revenues <- actual_revenues[complete.cases(actual_revenues),]
head(actual_revenues)

,Country,2010,2011,2012,2013,Total,Population,Density,long,lat,group,order
7726,Australia,1670,1500,1160,2,4332,25550683,3,123.5945,-12.42568,133,7115
7727,Australia,1670,1500,1160,2,4332,25550683,3,123.5952,-12.43594,133,7116
7728,Australia,1670,1500,1160,2,4332,25550683,3,123.5732,-12.43418,133,7117
7729,Australia,1670,1500,1160,2,4332,25550683,3,123.5725,-12.42393,133,7118
7730,Australia,1670,1500,1160,2,4332,25550683,3,123.5945,-12.42568,133,7119
8604,Australia,1670,1500,1160,2,4332,25550683,3,158.8788,-54.70976,139,7267


In [23]:
#merge normalized_revenue and world to get coordinate
normalized_revenues <- merge(normalized_revenues, world, by = 'Country', all='True')
normalized_revenues <- normalized_revenues[order(normalized_revenues$order),]
normalized_revenues <- normalized_revenues[complete.cases(normalized_revenues),]
head(normalized_revenues)

,Country,2010,2011,2012,2013,Total,Population,Density,long,lat,group,order
7726,Australia,65360.29,58706.85,45399.96,78.28,169545.4,25550683,3,123.5945,-12.42568,133,7115
7727,Australia,65360.29,58706.85,45399.96,78.28,169545.4,25550683,3,123.5952,-12.43594,133,7116
7728,Australia,65360.29,58706.85,45399.96,78.28,169545.4,25550683,3,123.5732,-12.43418,133,7117
7729,Australia,65360.29,58706.85,45399.96,78.28,169545.4,25550683,3,123.5725,-12.42393,133,7118
7730,Australia,65360.29,58706.85,45399.96,78.28,169545.4,25550683,3,123.5945,-12.42568,133,7119
8604,Australia,65360.29,58706.85,45399.96,78.28,169545.4,25550683,3,158.8788,-54.70976,139,7267


In [24]:
#convert from wide to long
actual_revenues_long <- melt(actual_revenues, id = 'Country', measure = c('2010', '2011', '2012', '2013'))
actual_revenues_long <- actual_revenues_long %>% rename('Year' = 'variable')
head(actual_revenues_long)

Country,Year,value
Australia,2010,1670
Australia,2010,1670
Australia,2010,1670
Australia,2010,1670
Australia,2010,1670
Australia,2010,1670


In [25]:
#convert from wide to long
normalized_revenues_long <- melt(normalized_revenues, id = 'Country', measure = c('2010', '2011', '2012', '2013'))
normalized_revenues_long <- normalized_revenues_long %>% rename('Year' = 'variable')
head(normalized_revenues_long)

Country,Year,value
Australia,2010,65360.29
Australia,2010,65360.29
Australia,2010,65360.29
Australia,2010,65360.29
Australia,2010,65360.29
Australia,2010,65360.29


In [26]:
# define shiny user interface
# define dashboard layout
ui <- navbarPage("Global Video Game Sales", theme = shinytheme("flatly"),
       #define first tab
       tabPanel("Sales Over Time",
                sidebarLayout(
                  #define settings
                  sidebarPanel(
                    style = 'background: ghostwhite',
                    h3('Settings'),
                      #define checkbox input
                    checkboxGroupInput("region", "Region", 
                                       choices = unique(normalized_sales_long$Region), 
                                       selected = unique(normalized_sales_long$Region)),
                    #define slider bar
                    sliderInput("year_range", "Year Range", 
                                min = 1979, max = 2019, value = 1979, step = 1, 
                                animate = animationOptions(500,loop = F), dragRange = T),
                    #add description
                    p('NA: North America'),
                    p('PAL: Asia, Africa, Europe, South America, Oceania'),
                    p('JP: Japan'),
                    p('Other: Other part of the world')
                  ),
                  #define visualizations area
                  mainPanel(
                    h3("Number of Sales Over Time", align='center'),
                    plotOutput('vis1'),
                    p('Data was collected from VGChartz.'),
                    p('The plots display the sales of video games in different regions of the world between 1979 and 2019. The sales 
                       were normalized by region populations. The decline in sales after 2008 can be due to the lack of raw data from 
                       the website (e.g., no relevant information of Nintendo Switch) and the rise of mobile games in recent years.'),
                    plotOutput('vis2')
                  )
              )
       ),
       #define second tab
       tabPanel("Countries Revenues",
                sidebarLayout(
                  #define settings
                  sidebarPanel(
                      style = 'background: ghostwhite',
                      h3('Settings'),
                      #define select input
                      selectInput("mode", "Mode", 
                                  choices = c('Normalized', 'Actual'),
                                  selected = 'Normalized'),
                      p('Actual: Raw revenues in billions US dollars'),
                      p('Normalized: Normalized revenues by billions population')),
                  #define visualization area
                  mainPanel(
                    plotlyOutput('vis3'),
                    p('Data was collected from Fandom.'),
                    p('The map above shows the average video game revenues made by different countries between 2010 and 2013, and tooltip is
                    enabled for searching for more information. Whereas, the barchart below displays the revenues each year. Japan has the 
                    highest revenues for both actual and normalized data.'),
                    plotOutput('vis4')
                  ))
          ),
       #define the third tab
       tabPanel("Sales by Categories",
                sidebarLayout(
                  #define settings
                  sidebarPanel(
                    style = 'background: ghostwhite',
                    h3('Settings'),
                    #define select input
                    selectInput('cat', 'Categories',
                                choices = c('Publisher', 'Platform', 'Genre'),
                                selected = 'Publisher'),
                    #define slider bar
                    sliderInput("range", "Range Bar",
                                min = 1, max = 60,
                                value = 15, step = 1),
                    #define text input for search
                    textInput('pub_name', 'Enter Publisher', 'Capcom'),
                    textInput('pla_name', 'Enter Platform', 'PS4'),
                    textInput('gen_name', 'Enter Genre', 'Action'),
                    p('Publisher: company that publishes video games.'),
                    p('Platform: console where video game can be played at.'),
                    p('Genre: genre of games (e.g., Action, Shooting, Sports).')
                  ),
                  #deine visualization
                  mainPanel(
                    plotOutput('vis5'),
                    p('Data was collected from VGChartz.'),
                    textOutput('text'),
                    plotlyOutput('vis6')
                    )
                  ))
       
       )

In [27]:
#create world map
world_map <- borders("world", colour="gray50", fill="white")
#define a function to plot the revenues on the map
revenue_map <- function(dataframe){
    ggplot(dataframe, aes(long, lat, group = group, 
                             label = Country,
                             label2 = Population)) + 
    world_map + 
    coord_sf(crs = st_crs('+proj=moll'),
             xlim = c(-180, 195), ylim = c(-60, 85),
             expand = F) + 
    theme_bw() + 
    geom_polygon(aes(fill = Total)) +
    scale_fill_viridis_c(option = "viridis") + 
    theme(legend.position="bottom") + 
    guides(size=FALSE) +
    ylab('Latitude') + xlab('Longitude')
}

In [28]:
#define a function to plot the revenue barchart
revenue_bar <- function(dataframe){
    ggplot(dataframe, aes(Country, value, fill=Year)) +
    geom_bar(stat='identity', position = "dodge", width = 0.8) + 
    theme_light(base_size = 12) + coord_flip() +
    theme(legend.position="bottom") +
    theme(axis.title.y=element_blank())
}

In [29]:
#define a function to plot the barchart for publisher, platform and genre
feature_plot <- function(feature, slide_point){
    temp <- actual_sales %>%
    group_by(!!enquo(feature)) %>%
    summarise(sales = sum(Global_Sales))
    temp <- temp[order(-temp$sales),]
    ggplot(temp[1:slide_point,], aes(reorder(!!enquo(feature), sales), sales, fill = sales)) + 
    geom_bar(stat='identity') + 
    ylab('Global Sales (million)') +
    theme_light(base_size = 12) + coord_flip() + 
    scale_fill_gradient(low = "yellow", high = "red", na.value = NA) +
    theme(axis.title.y = element_blank(), plot.title = element_text(size=22))
}

In [30]:
#define shiny server
server <- function(input, output, session) {
  #tab 1 first plot
  output$vis1 = renderPlot({
      normalized_sales_long %>% 
      filter(Region %in% input$region) %>%
      ggplot(aes(Year, value, col = Region)) + 
          xlim(1979, input$year_range) +
          geom_line() + theme_light() +
          theme(text = element_text(size = 15)) +
          ylab('Number of sales)') + xlab('Year')
  })
  
  #tab 1 second plot
  output$vis2 = renderPlot({
    normalized_sales_long %>%
      filter(Region %in% input$region) %>%
      ggplot(aes(Year, value, fill = Region)) +
      geom_bar(stat = 'identity') +
      theme_light() + theme(text = element_text(size = 15))+
      facet_wrap(~Region, 1, 5) + 
      ylab('Number of sales') + xlab('Year')
  })
  
  #tab 2 first plot
  output$vis3 = renderPlotly({
    if(input$mode == 'Normalized'){
      ggplotly(revenue_map(normalized_revenues) +  
               ggtitle('Normalized Revenue Distribution 2010 to 2013'))
    } else{
        ggplotly(revenue_map(Actual_revenues) +  
               ggtitle('Actual Revenue Distribution 2010 to 2013'))
    }
  })
  
  #tab 2 second plot
  output$vis4 = renderPlot({
    if(input$mode == 'Normalized'){
      revenue_bar(normalized_revenues_long) + ylab('Normalized Revenue')
    } else{
        revenue_bar(actual_revenues_long) + ylab('Actual Revenue')
    }
  })

  #tab 3 first plot
  output$vis5 = renderPlot({
      slide_point <- input$range
      if(input$cat == 'Publisher'){
          feature_plot(Publisher, slide_point) + ggtitle('Top Sales Publishers')
      } else if(input$cat == 'Platform'){
          feature_plot(Platform, slide_point) + ggtitle('Top Sales Platforms')
      } else{
          feature_plot(Genre, slide_point) + ggtitle('Top Sales Genres')
      }
  })
  
  #tab 3 description
  output$text = renderText({
    if(input$cat == 'Publisher'){
      'The bar chart above shows the overall global sales for different publishers in descending order. Whereas, the scatter plot below 
       displays the sales against published year for selected publisher.'
    } else if(input$cat == 'Platform'){
      'The bar chart above shows the overall global sales for different platforms in descending order. Whereas, the bar chart below displays
      the sales of different games for selected platform, and tooltip is enabled for searching more information.'
    } else {
      'The bar chart above shows the overall global sales for different genres in descending order. Whereas, the bar chart below displays
      the sales of different games for selected genre, and tooltip is enabled for searching more information. Moreover, some games were
      publsihed in different platformed as shown.'
    }
  })
    
  #tab 3 second plot
  #function is not defined for this plot as there are many differences within
  output$vis6 = renderPlotly({
    if(input$cat == 'Publisher'){
      temp <- actual_sales %>% filter(Publisher == input$pub_name) %>%
      select('Name', 'Year', 'Genre', 'Platform', 'Publisher', 'Global_Sales') %>%
      group_by(Publisher) %>%
      ggplot(aes(Year, Global_Sales, label = Name, label2 = Platform, label3 = Genre)) + 
      geom_point(aes(col = Global_Sales)) + ggtitle('Sales against Year') +
      ylab('Global sales (millions)') + theme_light()
      ggplotly(temp, tooltip = c('y', 'label', 'label2', 'label3'))
        
    } else if(input$cat == 'Platform'){
      temp <- actual_sales %>%
      filter(Platform == input$pla_name)
      temp <- temp[order(-temp$Global_Sales),]
      temp <- ggplot(temp[1:input$pub_range,], 
                     aes(reorder(Name, Global_Sales), Global_Sales, fill = Global_Sales, label = Year, label2 = Publisher)) + 
        geom_bar(stat='identity') + coord_flip() +
        ggtitle('Top Sales Games') + ylab('Global Sales (million)') + theme_light()
        ggplotly(temp, tooltip = c('y', 'label', 'label2'))
        
    } else{
      temp <- actual_sales %>% filter(Genre == input$gen_name)
      temp <- temp[order(-temp$Global_Sales),]
      temp <- ggplot(temp[1:input$pub_range,], 
                     aes(reorder(Name, Global_Sales), Global_Sales, fill = Global_Sales, label = Year, label2 = Platform)) + 
        geom_bar(stat='identity') + coord_flip() + ggtitle('Top Sales Games') + ylab('Global Sales (million)') + 
        theme_light() + theme(axis.title.y = element_blank())
      ggplotly(temp, tooltip = c('y', 'label', 'label2'))
    }
    
  })
  
}

In [ ]:
#call the shiny app
shinyApp(ui, server)


Listening on http://127.0.0.1:6875
Warning message:
"Removed 152 rows containing missing values (geom_path)."